### Imports

In [2]:
import requests
import json
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim import corpora


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maria\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maria\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Extracting entities with Babel
(Davide's code)
http://localhost:8888/notebooks/OneDrive/Documentos/Thesis/kick-off/NamedEntitiesRecognition-Copy1.ipynb

In [3]:
file_path = 'C:/Users/maria/OneDrive/Documentos/Thesis/Papers_Dataset/papers.csv'
papers_df = pd.read_csv(file_path)
papers_df

,title_s,journalTitle_s,authLastName_s,authFullName_s,abstract_s,authIdPerson_i,fileMain_s,ePublicationDateY_i,rteamStructAddress_s,Complete,Genders,content_text,cleaned_content_text,contentLen
0,"[""Exploring farmers' agrobiodiversity manageme...",People and Nature,"['Mariel', 'Carrière', 'Penot', 'Danthu', 'Raf...","['Juliette Mariel', 'Stéphanie M. Carrière', '...","[""1. Interactions between farmers and agrobiod...","[1099635, 746331, 762309, 799335]",https://hal.inrae.fr/hal-03461601/document,NaN,NaN,True,"['female', 'female', 'male', 'male', 'unknown'...",HAL Id: hal-03461601 https://hal.inrae.fr/hal-...,httpshalinraefrhal dec multidisciplinary open ...,48624
1,['Interpreting the mammal deposits of Cloggs C...,People and Nature,"['Mcdowell', 'David', 'Mullett', 'Fresløv', 'D...","['Matthew Mcdowell', 'Bruno David', 'Russell M...","[""Palaeontological animal bone deposits are ra...","[1179031, 1179032, 8412, 1179033, 1179034, 117...",https://hal.science/hal-03829927/document,NaN,NaN,True,"['male', 'male', 'male', 'female', 'unknown', ...",HAL Id: hal-03829927 https://hal.science/hal-0...,httpshalsciencehal mar multidisciplinary open ...,47840
2,['An ecological theory of changing human popul...,People and Nature,"['Henderson', 'Loreau']","['Kirsten Henderson', 'Michel Loreau']",['The dependence of humans on nature has come ...,[178300],https://hal.science/hal-02350603/document,NaN,NaN,True,"['female', 'male']",People and Nature. 2019;1:31–43. | 31 wileyonl...,people nature wileyonlinelibrarycomjournalpan ...,47839
3,"['Local knowledge, know‐how and knowledge mobi...",People and Nature,"['Congretel', 'Pinton']","['Mélanie Congretel', 'Florence F. Pinton']","[""More than 30 years after the 1988 Declaratio...",[175386],https://shs.hal.science/halshs-02937670/document,NaN,NaN,True,"['female', 'female']",HAL Id: halshs-02937670 https://shs.hal.scienc...,halshs httpsshshalsciencehalshs multidisciplin...,68534
4,['From biodiversity to health: Quantifying the...,People and Nature,"['Ulrich', 'Batáry', 'Baudry', 'Beaumelle', 'B...","['Werner Ulrich', 'Péter Batáry', 'Julia Baudr...",['Ample evidence suggests positive effects of ...,"[1250045, 772098, 736502, 1250046, 1250047, 12...",https://cnam.hal.science/hal-04080723/document,NaN,['Université Sorbonne Paris Nord- Bâtiment SMB...,True,"['male', 'male', 'female', 'female', 'male', '...",People and Nature. 2023;5:69–83. | 69 wileyonl...,people nature wileyonlinelibrarycomjournalpan ...,58890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124,['Genome structure and content of the rice roo...,Ecology and Evolution,"['Phan', 'Orjuela', 'Danchin', 'Klopp', 'Perfu...","['Ngan Thi Phan', 'Julie Orjuela', 'Etienne Da...","['Discovered in the 1960s,Meloidogyne graminic...","[734755, 735911, 1151095, 802414, 769869, 1098...",https://hal.science/hal-02988764/document,NaN,['Campus INRA24 chemin de borde rouge - Auzevi...,True,"['mostly_female', 'female', 'male', 'male', 'f...",HAL Id: hal-02988764 https://hal.science/hal-0...,httpshalsciencehal apr multidisciplinary open ...,59734
1125,['DNA barcoding of reef brittle stars (Ophiuro...,Ecology and Evolution,"['Boissin', 'Hoareau', 'Paulay', 'Bruggemann']","['Emilie Boissin', 'Thierry B Hoareau', 'Gusta...",['In anticipation of the current biodiversity ...,"[773657, 9775]",https://hal.univ-reunion.fr/hal-01907132/document,NaN,NaN,True,"['female', 'male', 'male', 'male']",HAL Id: hal-01907132 https://hal.univ-reunion....,httpshalunivreunionfrhal oct multidisciplinary...,24089
1126,['Adaptation and plasticity in aboveground all...,Ecology and Evolution,"['Vizcaíno-Palomar', 'Ibáñez', 'González-Martí...","['Natalia Vizcaíno-Palomar', 'Inés Ibáñez', 'S...","[""Plant species aboveground allometry can be v...",[747477],https://hal.inrae.fr/hal-02636091/document,NaN,NaN,True,"['female', 'female', 'male', 'male', 'male']",HAL Id: hal-02636091 https://hal.inrae.fr/hal-...,httpshalinraefrhal multidisciplinary open acce...,36649
1127,['Unidi

In [21]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")

def extract_entities(df, text_column):
    #Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
    model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")

    #Define chunk size
    max_length = tokenizer.model_max_length
    chunk_size = max_length

    #Define NER pipeline
    nlp = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)

    df['entities'] = None

    #Loop over each row in the dataframe
    for index, row in df.iterrows():
        #Clean the text
        clean_text = row[text_column]

        #Create an empty list to store the entities
        entity_list = []

        #Split the text into chunks
        chunks = [clean_text[i:i+chunk_size] for i in range(0, len(clean_text), chunk_size)]

        #Perform NER on each chunk and extract the entities
        for chunk in chunks:
            results = nlp(chunk)
            for result in results:
                entity_list.append(f"{result['word']} ({result['entity_group']})")

        df.at[index, 'entities'] = entity_list

    return df

papers_entities_df = extract_entities(papers_df, 'cleaned_content_text')

C:\Users\maria\anaconda3\lib\site-packages\transformers\pipelines\token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


KeyboardInterrupt: 

In [4]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import time

tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")

def extract_entities(df, text_column):
    print("Loading tokenizer and model...")
    tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
    model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")

    # Define chunk size
    max_length = tokenizer.model_max_length
    chunk_size = max_length

    # Define NER pipeline
    print("Initializing NER pipeline...")
    nlp = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)

    df['entities'] = None

    total_rows = len(df)
    print(f"Processing {total_rows} rows...")
    
    start_time = time.time()

    # Loop over each row in the dataframe
    for index, row in df.iterrows():
        # Clean the text
        clean_text = row[text_column]
        print("Entered for 1")

        # Create an empty list to store the entities
        entity_list = []

        # Split the text into chunks
        chunks = [clean_text[i:i+chunk_size] for i in range(0, len(clean_text), chunk_size)]

        # Perform NER on each chunk and extract the entities
        for chunk in chunks:
            results = nlp(chunk)
            for result in results:
                entity_list.append(f"{result['word']} ({result['entity_group']})")
                print(f"{result['word']} ({result['entity_group']})")

        df.at[index, 'entities'] = entity_list

        # Print progress every 100 rows
        if (index + 1) % 100 == 0:
            elapsed_time = time.time() - start_time
            print(f"Processed {index + 1}/{total_rows} rows. Elapsed time: {elapsed_time:.2f} seconds")

    return df

# Assuming papers_df and 'cleaned_content_text' are correctly defined
papers_entities_df = extract_entities(papers_df, 'cleaned_content_text')



Loading tokenizer and model...
Initializing NER pipeline...
Processing 1129 rows...
Entered for 1


C:\Users\maria\anaconda3\lib\site-packages\transformers\pipelines\token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


Entered for 3
##sa (LOC)
Entered for 3
##bo (MISC)
Entered for 3
mala (MISC)
Entered for 3
##gasy (MISC)
Entered for 3
##riat (LOC)
Entered for 3
##sitohaina (LOC)
Entered for 3
et (LOC)
Entered for 3
et (LOC)
Entered for 1
Entered for 3
##ai (LOC)
Entered for 3
gun (LOC)
Entered for 3
##aikurnai (LOC)
Entered for 3
##ai (LOC)
Entered for 3
aust (LOC)
Entered for 1
Entered for 3
eberstadt (LOC)
Entered for 3
ga (PER)
Entered for 3
s (MISC)
Entered for 3
##can (MISC)
Entered for 3
##dina (MISC)
Entered for 3
##vian (MISC)
Entered for 3
sub (LOC)
Entered for 3
##aran (LOC)
Entered for 3
e (LOC)
Entered for 3
hende (PER)
Entered for 1
Entered for 3
berthefriedberg (PER)
Entered for 3
i (MISC)
Entered for 3
e (MISC)
Entered for 3
hol (MISC)
Entered for 3
kloppenburg jr (PER)
Entered for 1
Entered for 3
uma (MISC)
Entered for 3
fa (PER)
Entered for 3
##holm (PER)
Entered for 3
antos martín (PER)
Entered for 3
gar (LOC)
Entered for 3
garibaldi (PER)
Entered for 3
ever (MISC)
Entered for 3
u 

Entered for 1
Entered for 3
i (ORG)
Entered for 3
ina (MISC)
Entered for 1
Entered for 3
ological (MISC)
Entered for 1
Entered for 3
india (LOC)
Entered for 3
un (LOC)
Entered for 1
Entered for 3
fr (MISC)
Entered for 1
Entered for 3
n (MISC)
Entered for 3
europea (MISC)
Entered for 3
go (PER)
Entered for 3
europea (MISC)
Entered for 3
lamersdorf (PER)
Entered for 3
andreas schuldt (PER)
Entered for 1
Entered for 3
cal (LOC)
Entered for 3
##orn (LOC)
Entered for 3
##ynésie française (ORG)
Entered for 3
mo (LOC)
Entered for 3
##as (LOC)
Entered for 3
ur (MISC)
Entered for 3
vedriven (LOC)
Entered for 3
ng (LOC)
Entered for 1
Entered for 3
ku (LOC)
Entered for 3
unisol (ORG)
Entered for 3
i (LOC)
Entered for 3
mula (LOC)
Entered for 3
ku (PER)
Entered for 3
##akov (PER)
Entered for 3
dit (MISC)
Entered for 1
Entered for 3
le (MISC)
Entered for 3
it (MISC)
Entered for 3
fondation recherche biodiversite (ORG)
Entered for 1
Entered for 3
gan (MISC)
Entered for 3
k (LOC)
Entered for 3
##vern

Entered for 3
bi (LOC)
Entered for 3
##wie (LOC)
Entered for 1
Entered for 3
ministère (ORG)
Entered for 3
ai (LOC)
Entered for 1
Entered for 3
dis (MISC)
Entered for 3
ve (PER)
Entered for 3
mae (LOC)
Entered for 3
##pe (LOC)
Entered for 3
fi (MISC)
Entered for 1
Entered for 3
mil (MISC)
Entered for 3
lt (PER)
Entered for 3
neo (PER)
Entered for 3
uit (MISC)
Entered for 1
Entered for 1
Entered for 3
en (LOC)
Entered for 3
##khtu (LOC)
Entered for 3
wil (ORG)
Entered for 1
Entered for 3
re (MISC)
Entered for 3
ming (MISC)
Entered for 1
Entered for 3
montsegur (LOC)
Entered for 3
az (LOC)
Entered for 3
es (MISC)
Entered for 3
vanreusel (PER)
Entered for 3
az (LOC)
Entered for 1
Entered for 3
https (ORG)
Entered for 3
et (LOC)
Entered for 3
gr (PER)
Entered for 3
et (LOC)
Entered for 3
##hiopia (LOC)
Entered for 3
et (LOC)
Entered for 3
##hiopia (LOC)
Entered for 3
c (PER)
Entered for 3
##wley (PER)
Entered for 3
azt (MISC)
Entered for 3
ich (MISC)
Entered for 3
##er (MISC)
Entered for 1

...

Entered for 3
ro (MISC)
Entered for 3
nm (LOC)
Entered for 3
ro (MISC)
Entered for 3
kearney (PER)
Entered for 3
eo (MISC)
Entered for 3
dot (MISC)
Entered for 3
tat (MISC)
Entered for 3
and (PER)
Entered for 1
Entered for 3
santi (LOC)
Entered for 3
kounosu (PER)
Entered for 3
ci (MISC)
Entered for 1
Entered for 3
ki (LOC)
Entered for 3
k (LOC)
Entered for 3
##édougou (LOC)
Entered for 3
ni (LOC)
Entered for 3
##ba (LOC)
Entered for 3
k (LOC)
Entered for 3
##gou (LOC)
Entered for 3
##counda (LOC)
Entered for 3
niger (LOC)
Entered for 3
k (LOC)
Entered for 3
##gou (LOC)
Entered for 3
him (LOC)
Entered for 3
##öhle (PER)
Entered for 3
##gou (LOC)
Entered for 3
##nda (LOC)
Processed 900/1129 rows. Elapsed time: 19576.14 seconds
Entered for 1
Entered for 3
john wil (PER)
Entered for 3
huelsenbeck (PER)
Entered for 3
john wil (PER)
Entered for 3
jo (PER)
Entered for 3
wil (PER)
Entered for 3
##tropical (MISC)
Entered for 3
john wiley (PER)
Entered for 3
jo (PER)
Entered for 3
wil (PER)
Ent

Entered for 3
##wieża (LOC)
Entered for 1
Entered for 1
Entered for 3
jo (PER)
Entered for 3
wil (PER)
Entered for 3
jo (PER)
Entered for 3
wil (PER)
Entered for 3
jo (PER)
Entered for 3
wil (PER)
Entered for 3
jo (PER)
Entered for 3
wil (PER)
Entered for 3
##fordshire (LOC)
Entered for 3
st (MISC)
Entered for 3
jo (PER)
Entered for 3
wil (PER)
Entered for 3
jo (PER)
Entered for 3
wil (PER)
Entered for 3
e (MISC)
Entered for 3
john wil (PER)
Entered for 1
Entered for 3
res (MISC)
Entered for 3
##usseau (PER)
Entered for 3
paaby (PER)
Entered for 3
pa (LOC)
Entered for 3
vel (MISC)
Entered for 3
e (MISC)
Entered for 3
row (PER)
Entered for 3
net (MISC)
Entered for 3
pa (LOC)
Entered for 3
pa (LOC)
Entered for 3
pa (LOC)
Entered for 3
gi (MISC)
Entered for 3
https (MISC)
Entered for 1
Entered for 3
europea (MISC)
Entered for 3
euro (LOC)
Entered for 3
e (ORG)
Entered for 3
idi (MISC)
Entered for 1
Entered for 3
hat (MISC)
Entered for 3
bornholm (LOC)
Entered for 3
nito (MISC)
Entered for

Entered for 1
Entered for 3
jo (PER)
Entered for 3
wil (PER)
Entered for 3
université paris (LOC)
Entered for 3
université paris (LOC)
Entered for 3
jean david (PER)
Entered for 3
tome (LOC)
Entered for 3
##lić ve (PER)
Entered for 1
Entered for 3
##vaux (PER)
Entered for 1
Entered for 3
cheverud (MISC)
Entered for 3
mart (PER)
Entered for 3
mil (MISC)
Entered for 3
ga (PER)
Entered for 3
ona (MISC)
Entered for 3
ne (MISC)
Entered for 3
lande (PER)
Entered for 3
e (MISC)
Entered for 3
heidelberg (PER)
Entered for 1
Entered for 3
e (MISC)
Entered for 3
ri (PER)
Entered for 3
jardin (LOC)
Entered for 3
jardin (LOC)
Entered for 3
har (MISC)
Entered for 3
##guin (MISC)
Entered for 1
Entered for 3
jo (PER)
Entered for 3
wil (PER)
Entered for 3
jo (PER)
Entered for 3
benjamini (PER)
Entered for 3
ay (MISC)
Entered for 3
og (MISC)
Entered for 1
Entered for 3
st (PER)
Entered for 3
##berg (PER)
Entered for 3
hess (PER)
Entered for 3
##agawa (PER)
Entered for 3
nak (MISC)
Entered for 3
##agawa 

Entered for 1
Entered for 3
eso (MISC)
Entered for 3
jo (PER)
Entered for 3
wil (PER)
Entered for 3
eso (MISC)
Entered for 3
jo (PER)
Entered for 3
wil (PER)
Entered for 3
ras (MISC)
Entered for 3
i (MISC)
Entered for 3
m (MISC)
Entered for 1
Entered for 3
caledonia (LOC)
Entered for 3
cal (LOC)
Entered for 3
jo (PER)
Entered for 3
jo (PER)
Entered for 3
##sa (LOC)
Entered for 1
Entered for 3
westeberhard (PER)
Entered for 3
jo (PER)
Entered for 3
nt (MISC)
Entered for 3
jo (PER)
Entered for 3
ass (MISC)
Entered for 3
john wiley (PER)
Entered for 3
en (MISC)
Entered for 3
ng (MISC)
Entered for 3
o (PER)
Entered for 1
Entered for 3
che (LOC)
Entered for 3
##mno (LOC)
Entered for 3
i (MISC)
Entered for 3
ni (LOC)
Entered for 3
bra (LOC)
Entered for 1
Entered for 3
##chwein (LOC)
Entered for 3
##pierre (LOC)
Entered for 3
s (LOC)
Entered for 3
##weinsberg (LOC)
Entered for 3
s (LOC)
Entered for 3
##sberg (LOC)
Entered for 1
Entered for 3
jo (PER)
Entered for 3
wil (PER)
Entered for 3
jo (

Entered for 1
Processed 1100/1129 rows. Elapsed time: 23184.23 seconds
Entered for 1
Entered for 3
bu (PER)
Entered for 3
lu (MISC)
Entered for 3
e (MISC)
Entered for 3
ig (MISC)
Entered for 3
aj (MISC)
Entered for 1
Entered for 3
download (MISC)
Entered for 1
Entered for 3
black (ORG)
Entered for 1
Entered for 3
istri (MISC)
Entered for 3
em (MISC)
Entered for 3
m (LOC)
Entered for 3
ng (MISC)
Entered for 3
te (PER)
Entered for 3
bryden (PER)
Entered for 3
m (LOC)
Entered for 3
adélie (PER)
Entered for 1
Entered for 3
k (LOC)
Entered for 3
##öppen (LOC)
Entered for 3
but (LOC)
Entered for 3
nt (MISC)
Entered for 3
##erface (MISC)
Entered for 3
##öppen (LOC)
Entered for 3
io (MISC)
Entered for 1
Entered for 3
pt (MISC)
Entered for 3
john wiley (PER)
Entered for 3
s (PER)
Entered for 3
##malt (PER)
Entered for 3
jo (PER)
Entered for 3
wil (PER)
Entered for 3
##v (PER)
Entered for 3
john wiley (PER)
Entered for 3
john wil (PER)
Entered for 3
ele (PER)
Entered for 1
Entered for 3
https (M

In [17]:
papers_entities_df.to_csv('C:/Users/maria/OneDrive/Documentos/Thesis/paper_entities_dataset2.csv', index=False)

In [6]:
papers_entities_df

,title_s,journalTitle_s,authLastName_s,authFullName_s,abstract_s,authIdPerson_i,fileMain_s,ePublicationDateY_i,rteamStructAddress_s,Complete,Genders,content_text,cleaned_content_text,contentLen,entities
0,"[""Exploring farmers' agrobiodiversity manageme...",People and Nature,"['Mariel', 'Carrière', 'Penot', 'Danthu', 'Raf...","['Juliette Mariel', 'Stéphanie M. Carrière', '...","[""1. Interactions between farmers and agrobiod...","[1099635, 746331, 762309, 799335]",https://hal.inrae.fr/hal-03461601/document,NaN,NaN,True,"['female', 'female', 'male', 'male', 'unknown'...",HAL Id: hal-03461601 https://hal.inrae.fr/hal-...,httpshalinraefrhal dec multidisciplinary open ...,48624,"[##sa (LOC), ##bo (MISC), mala (MISC), ##gasy ..."
1,['Interpreting the mammal deposits of Cloggs C...,People and Nature,"['Mcdowell', 'David', 'Mullett', 'Fresløv', 'D...","['Matthew Mcdowell', 'Bruno David', 'Russell M...","[""Palaeontological animal bone deposits are ra...","[1179031, 1179032, 8412, 1179033, 1179034, 117...",https://hal.science/hal-03829927/document,NaN,NaN,True,"['male', 'male', 'male', 'female', 'unknown', ...",HAL Id: hal-03829927 https://hal.science/hal-0...,httpshalsciencehal mar multidisciplinary open ...,47840,"[##ai (LOC), gun (LOC), ##aikurnai (LOC), ##ai..."
2,['An ecological theory of changing human popul...,People and Nature,"['Henderson', 'Loreau']","['Kirsten Henderson', 'Michel Loreau']",['The dependence of humans on nature has come ...,[178300],https://hal.science/hal-02350603/document,NaN,NaN,True,"['female', 'male']",People and Nature. 2019;1:31–43. | 31 wileyonl...,people nature wileyonlinelibrarycomjournalpan ...,47839,"[eberstadt (LOC), ga (PER), s (MISC), ##can (M..."
3,"['Local knowledge, know‐how and knowledge mobi...",People and Nature,"['Congretel', 'Pinton']","['Mélanie Congretel', 'Florence F. Pinton']","[""More than 30 years after the 1988 Declaratio...",[175386],https://shs.hal.science/halshs-02937670/document,NaN,NaN,True,"['female', 'female']",HAL Id: halshs-02937670 https://shs.hal.scienc...,halshs httpsshshalsciencehalshs multidisciplin...,68534,"[berthefriedberg (PER), i (MISC), e (MISC), ho..."
4,['From biodiversity to health: Quantifying the...,People and Nature,"['Ulrich', 'Batáry', 'Baudry', 'Beaumelle', 'B...","['Werner Ulrich', 'Péter Batáry', 'Julia Baudr...",['Ample evidence suggests positive effects of ...,"[1250045, 772098, 736502, 1250046, 1250047, 12...",https://cnam.hal.science/hal-04080723/document,NaN,['Université Sorbonne Paris Nord- Bâtiment SMB...,True,"['male', 'male', 'female', 'female', 'male', '...",People and Nature. 2023;5:69–83. | 69 wileyonl...,people nature wileyonlinelibrarycomjournalpan ...,58890,"[uma (MISC), fa (PER), ##holm (PER), antos mar..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124,['Genome structure and content of the rice roo...,Ecology and Evolution,"['Phan', 'Orjuela', 'Danchin', 'Klopp', 'Perfu...","['Ngan Thi Phan', 'Julie Orjuela', 'Etienne Da...","['Discovered in the 1960s,Meloidogyne graminic...","[734755, 735911, 1151095, 802414, 769869, 1098...",https://hal.science/hal-02988764/document,NaN,['Campus INRA24 chemin de borde rouge - Auzevi...,True,"['mostly_female', 'female', 'male', 'male', 'f...",HAL Id: hal-02988764 https://hal.science/hal-0...,httpshalsciencehal apr multidisciplinary open ...,59734,[ro (PER)]
1125,['DNA barcoding of reef brittle stars (Ophiuro...,Ecology and Evolution,"['Boissin', 'Hoareau', 'Paulay', 'Bruggemann']","['Emilie Boissin', 'Thierry B Hoareau', 'Gusta...",['In anticipation of the current biodiversity ...,"[773657, 9775]",https://hal.univ-reunion.fr/hal-01907132/document,NaN,NaN,True,"['female', 'male', 'male', 'male']",HAL Id: hal-01907132 https://hal.univ-reunion....,httpshalunivreunionfrhal oct multidisciplinary...,24089,"[flor (LOC), e (MISC), https (MISC)]"
1126,['Adaptation and plasticity in aboveground all...,Ecology and Evolution,"['Vizcaíno-Palomar', 'Ibáñez', 'González-Martí...","['Natalia V

## Get Unique Entities

In [12]:
from collections import Counter

entities_df = papers_entities_df.copy()
#Create an empty list to store all entities
all_entities = []

#Iterate over the 'entities' column and append all entities to the list
for entities in entities_df['entities']:
    for entity in entities:
        all_entities.append(entity)

#Create a Counter object from the list of entities
entity_counts = Counter(all_entities)

#Create a new DataFrame with the entity counts
webpage_NamedEntities = pd.DataFrame({'entity': entity_counts.keys(), 'count': entity_counts.values()})

#Sort the DataFrame by count in descending order
webpage_NamedEntities = webpage_NamedEntities.sort_values(by='count', ascending=False)

webpage_NamedEntities = webpage_NamedEntities.reset_index(drop=True)
webpage_NamedEntities

,entity,count
0,jo (PER),350
1,wil (PER),243
2,e (MISC),82
3,john wiley (PER),56
4,euro (LOC),53
...,...,...
2447,suzukii (LOC),1
2448,w (MISC),1
2449,##rski (PER),1
2450,wileyblackwell (MISC),1


In [19]:
webpage_NamedEntities.to_csv('C:/Users/maria/OneDrive/Documentos/Thesis/entities_dataset2.csv', index=False, encoding = 'utf-8-sig')

In [14]:
webpage_NamedEntities

,entity,count,type
0,jo (PER),350,PER
1,wil (PER),243,PER
2,e (MISC),82,MISC
3,john wiley (PER),56,PER
4,euro (LOC),53,LOC
...,...,...,...
2447,suzukii (LOC),1,LOC
2448,w (MISC),1,MISC
2449,##rski (PER),1,PER
2450,wileyblackwell (MISC),1,MISC


In [13]:
#Extract the entity type from the "entity" column
webpage_NamedEntities['type'] = webpage_NamedEntities['entity'].str.split('(', 1).str[1].str[:-1]

#Count the occurrences of each entity type
entity_counts = webpage_NamedEntities['type'].value_counts()

#Xreate a new dataframe with entity type and count
entity_counts_df = pd.DataFrame({'entity type': entity_counts.index, 'count': entity_counts.values})

entity_counts_df

C:\Users\maria\AppData\Local\Temp\ipykernel_22124\2371940661.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  webpage_NamedEntities['type'] = webpage_NamedEntities['entity'].str.split('(', 1).str[1].str[:-1]


,entity type,count
0,LOC,839
1,MISC,755
2,PER,743
3,ORG,115


# TaxoNERD

https://github.com/nleguillarme/taxonerd/blob/main/README.md#use-as-python-module